# Distribuições

Pense Bayes, Segunda Edição

Copyright 2020 Allen B. Downey

Licença: [Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0)](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# If we're running on Colab, install empiricaldist
# https://pypi.org/project/empiricaldist/

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install empiricaldist

In [2]:
# Get utils.py

from os.path import basename, exists

def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve
        local, _ = urlretrieve(url, filename)
        print('Downloaded ' + local)
    
download('https://github.com/AllenDowney/ThinkBayes2/raw/master/soln/utils.py')

In [3]:
from utils import set_pyplot_params
set_pyplot_params()

No capítulo anterior utilizámos o Teorema de Bayes para resolver um problema de biscoitos; depois resolvemo-lo novamente utilizando uma mesa de Bayes.

Neste capítulo, correndo o risco de testar a sua paciência, resolvê-lo-emos mais uma vez utilizando um objecto `Pmf`, que representa uma "função de massa de probabilidade".

Vou explicar o que isso significa, e porque é útil para as estatísticas Bayesianas.

Vamos utilizar objectos `Pmf` para resolver alguns problemas mais desafiantes e dar mais um passo em direcção às estatísticas Bayesianas.

Mas vamos começar com as distribuições.

## Distribuições

Nas estatísticas uma **distribuição** é um conjunto de resultados possíveis e as suas correspondentes probabilidades.

Por exemplo, se atirar uma moeda ao ar, há dois resultados possíveis com

probabilidade aproximadamente igual.

Se lançar um dado de seis lados, o conjunto de resultados possíveis são os números 1 a 6, e a probabilidade associada a cada resultado é de 1/6.

Para representar as distribuições, utilizaremos uma biblioteca chamada `empiricaldistribuidora'.

Uma distribuição "empírica" é baseada em dados, em oposição a uma

distribuição teórica.

Vamos utilizar esta biblioteca ao longo de todo o livro.  Introduzirei as características básicas neste capítulo e veremos características adicionais mais tarde.

## Funções de probabilidade de massa


"Empiricaldist" fornece uma classe chamada "Pmf" que representa um

função massa de probabilidade.

Para utilizar `Pmf` pode importá-lo assim:

In [4]:
from empiricaldist import Pmf

Se isso não funcionar, poderá ter de instalar "empiricalista"; tente correr

```
!pip install empiricaldist
```

numa célula de código ou

```
pip install empiricaldist
```

numa janela terminal.

O exemplo seguinte faz um `Pmf` que representa o resultado de um

atirar moeda ao ar.

In [5]:
coin = Pmf()
coin['heads'] = 1/2
coin['tails'] = 1/2
coin

O `Pmf` cria um `Pmf` vazio, sem resultados.

Depois podemos adicionar novos resultados usando o operador de parênteses.



Também se pode fazer um `Pmf` a partir de uma sequência de possíveis resultados.

O exemplo seguinte utiliza `Pmf.from_seq` para fazer um `Pmf` que representa um dado de seis lados.

In [6]:
die = Pmf.from_seq([1,2,3,4,5,6])
die

Neste exemplo, todos os resultados da sequência aparecem uma vez, pelo que todos têm a mesma probabilidade, $1/6$.

Em termos mais gerais, os resultados podem aparecer mais do que uma vez, como no exemplo seguinte:

In [7]:
letters = Pmf.from_seq(list('Mississippi'))
letters


A letra `i` aparece 4 vezes, pelo que a sua probabilidade é de $4/11$.

Uma vez que as letras numa cadeia não são resultados de um processo aleatório, vou utilizar o termo mais geral "quantidades" para as letras no `Pmf`.

A classe `Pmf` herda de uma `Série Pandas`, por isso qualquer coisa que se possa fazer com uma `Série`, também se pode fazer com uma `Pmf`.

Por exemplo, pode utilizar o operador de parênteses para procurar uma quantidade e obter a probabilidade correspondente.

In [8]:
letters['s']

Na palavra "Mississippi", cerca de 36% das letras são "s".

No entanto, se pedir a probabilidade de uma quantidade que não esteja na distribuição, recebe um "KeyError".



In [9]:
try:
    letters['t']
except KeyError as e:
    print(type(e))

Também se pode chamar um `Pmf` como se fosse uma função, com uma letra entre parênteses.

In [10]:
letters('s')

Se a quantidade estiver na distribuição, os resultados são os mesmos. 

Mas se não estiver na distribuição, o resultado é `0`, não é um erro.

In [11]:
letters('t')

In [12]:
die([1,4,7])

As quantidades numa `Pmf` podem ser cordas, números, ou qualquer outro tipo que possa ser armazenado no índice de uma `Série Pandas`.

Se estiver familiarizado com Pandas, isso ajudá-lo-á a trabalhar com objectos `Pmf`. 

Mas vou explicar-lhe o que precisa de saber à medida que formos avançando.

## O Problema do Cookie Revisitado

Nesta secção vou utilizar um `Pmf` para resolver o problema do cookie de <<_TheCookieProblem>>>>.

Aqui está novamente a declaração do problema:

> Suponha que existem duas tigelas de biscoitos.

>

> Taça 1 contém 30 biscoitos de baunilha e 10 biscoitos de chocolate. 

>

> * Taça 2 contém 20 biscoitos de baunilha e 20 biscoitos de chocolate.

>

> Agora suponha que escolhe uma das taças ao acaso e, sem olhar, escolhe uma bolacha ao acaso. Se a bolacha for de baunilha, qual é a probabilidade de ter vindo da Taça 1?

Aqui está um `Pmf` que representa as duas hipóteses e as suas probabilidades anteriores:

In [13]:
prior = Pmf.from_seq(['Bowl 1', 'Bowl 2'])
prior

Esta distribuição, que contém a probabilidade prévia para cada hipótese, é chamada (esperar por ela) a **distribuição de prioridades***.

Para actualizar a distribuição com base em novos dados (o cookie baunilha),

multiplicamos os antecedentes pelas probabilidades. A probabilidade

de desenhar um biscoito de baunilha do Bowl 1 é `3/4`. A probabilidade de

para o Bowl 2 é `1/2`.

In [14]:
likelihood_vanilla = [0.75, 0.5]
posterior = prior * likelihood_vanilla
posterior

O resultado são os posteriors não normalizados; ou seja, não somam 1.

Para as fazer somar até 1, podemos utilizar "normalizar", que é um método fornecido por "Pmf".

In [15]:
posterior.normalize()

O valor de retorno de `normalizar` é a probabilidade total dos dados, que é $5/8$.

O "pósterior", que contém a probabilidade posterior para cada hipótese, chama-se (aguarde agora) a **distribuição pósterior**.

In [16]:
posterior

A partir da distribuição posterior podemos seleccionar a probabilidade posterior para a Taça 1:

In [17]:
posterior('Bowl 1')

E a resposta é 0,6.

Uma das vantagens de utilizar objectos `Pmf` é que é fácil fazer actualizações sucessivas com mais dados.

Por exemplo, suponha que volta a colocar o primeiro biscoito (para que o conteúdo das tigelas não mude) e desenha novamente da mesma tigela.

Se o segundo cookie for também de baunilha, podemos fazer uma segunda actualização como esta:

In [18]:
posterior *= likelihood_vanilla
posterior.normalize()
posterior

Agora a probabilidade posterior para o Bowl 1 é de quase 70%.

Mas suponhamos que fazemos a mesma coisa novamente e recebemos um biscoito de chocolate.

Aqui estão as probabilidades para os novos dados:

In [19]:
likelihood_chocolate = [0.25, 0.5]

E aqui está a actualização.

In [20]:
posterior *= likelihood_chocolate
posterior.normalize()
posterior

Agora a probabilidade posterior para a Taça 1 é de cerca de 53%.

Após dois biscoitos de baunilha e um de chocolate, as probabilidades posteriores são próximas de 50/50.

## 101 Taças

A seguir vamos resolver um problema de bolachas com 101 tigelas:

* Tigela 0 contém 0% de biscoitos de baunilha,

* Taça 1 contém 1% de biscoitos de baunilha,

* Tigela 2 contém 2% de biscoitos de baunilha,


* Tigela 99 contém 99% de biscoitos de baunilha, e

* Bowl 100 contém todos os biscoitos de baunilha.

Tal como na versão anterior, existem apenas dois tipos de biscoitos, baunilha e chocolate.  Portanto, Bowl 0 é todo biscoitos de chocolate, Bowl 1 é 99% de chocolate, e assim por diante.

Suponhamos que escolhemos uma tigela ao acaso, escolhemos um biscoito ao acaso, e acaba por ser baunilha.  Qual é a probabilidade de o biscoito ter vindo da tigela $x$, para cada valor de $x$?



In [21]:
import numpy as np

hypos = np.arange(101)

Podemos utilizar esta matriz para fazer a distribuição prévia:

In [22]:
prior = Pmf(1, hypos)
prior.normalize()

Como este exemplo mostra, podemos inicializar um `Pmf` com dois parâmetros.

O primeiro parâmetro é a probabilidade prévia; o segundo parâmetro é uma sequência de quantidades.

Neste exemplo, as probabilidades são todas iguais, pelo que só temos de fornecer uma delas; ela é "transmitida" através das hipóteses.

Como todas as hipóteses têm a mesma probabilidade anterior, esta distribuição é **uniforme***.

Aqui estão as primeiras hipóteses e as suas probabilidades.

In [23]:
prior.head()

A probabilidade dos dados é a fracção de biscoitos de baunilha em cada tigela, que podemos calcular utilizando "hypos":

In [24]:
likelihood_vanilla = hypos/100
likelihood_vanilla[:5]

Agora podemos calcular a distribuição posterior da forma habitual:



In [25]:
posterior1 = prior * likelihood_vanilla
posterior1.normalize()
posterior1.head()

A figura seguinte mostra a distribuição prévia e a distribuição posterior após um biscoito de baunilha.

In [26]:
from utils import decorate

def decorate_bowls(title):
    decorate(xlabel='Bowl #',
             ylabel='PMF',
             title=title)

In [27]:
prior.plot(label='prior', color='C5')
posterior1.plot(label='posterior', color='C4')
decorate_bowls('Posterior after one vanilla cookie')

A probabilidade posterior de Bowl 0 é 0 porque não contém biscoitos de baunilha.

A probabilidade posterior de Bowl 100 é a mais alta porque contém a maior quantidade de biscoitos de baunilha.

No meio, a forma da distribuição posterior é uma linha porque as probabilidades são proporcionais aos números das tigelas.

Agora suponhamos que voltamos a colocar o biscoito, tiramos de novo da mesma tigela, e recebemos outro biscoito de baunilha.

Aqui está a actualização após o segundo cookie:

In [28]:
posterior2 = posterior1 * likelihood_vanilla
posterior2.normalize()

E aqui está como se parece a distribuição posterior.

In [29]:
posterior2.plot(label='posterior', color='C4')
decorate_bowls('Posterior after two vanilla cookies')

Após dois biscoitos de baunilha, as tigelas de números altos têm as probabilidades posteriores mais elevadas porque contêm o maior número de biscoitos de baunilha; as tigelas de números baixos têm as probabilidades mais baixas.

Mas suponhamos que voltamos a desenhar e recebemos um biscoito de chocolate.



In [30]:
likelihood_chocolate = 1 - hypos/100

posterior3 = posterior2 * likelihood_chocolate
posterior3.normalize()

E aqui está a distribuição posterior.

In [31]:
posterior3.plot(label='posterior', color='C4')
decorate_bowls('Posterior after 2 vanilla, 1 chocolate')

Agora a Bowl 100 foi eliminada porque não contém biscoitos de chocolate.

Mas as tigelas de números altos ainda são mais prováveis do que as de números baixos, porque já vimos mais bolachas de baunilha do que de chocolate.

De facto, o pico da distribuição posterior é no Bowl 67, o que corresponde à fracção de biscoitos de baunilha nos dados que observámos, $2/3$.

A quantidade com a maior probabilidade posterior chama-se **MAP***, que significa "maximum a posteori probability", onde "a posteori" é latim desnecessário para "posterior".

Para calcular o MAP, podemos utilizar o método `Series` `idxmax`:

In [32]:
posterior3.idxmax()

Ou `Pmf` fornece um nome mais memorável para a mesma coisa:

In [33]:
posterior3.max_prob()

Como se pode suspeitar, este exemplo não se trata realmente de taças; trata-se de estimar as proporções.

Imagine que tem uma tigela de biscoitos.

Não sabe que fracção de biscoitos são de baunilha, mas pensa que é igualmente provável que seja qualquer fracção de 0 a 1.

Se desenhar três bolachas e duas são de baunilha, que proporção de bolachas na tigela pensa que são de baunilha?

A distribuição posterior que acabámos de calcular é a resposta a essa pergunta.

Voltaremos a estimar as proporções no próximo capítulo.

Mas primeiro vamos utilizar um `Pmf` para resolver o problema dos dados.

## O problema dos dados

No capítulo anterior, resolvemos o problema dos dados utilizando uma tabela Bayes.


> Suponha que tenho uma caixa com um dado de 6 faces, um dado de 8 faces, e um dado de 12 faces.

> Escolho um dos dados ao acaso, lanço-o, e relato que o resultado é um 1.

> Qual é a probabilidade de eu ter escolhido o dado de 6 lados?

Vamos resolvê-lo utilizando um `Pmf`.

Vou usar números inteiros para representar as hipóteses:

In [34]:
hypos = [6, 8, 12]

Podemos fazer a distribuição prévia desta forma:



In [35]:
prior = Pmf(1/3, hypos)
prior

Tal como no exemplo anterior, a probabilidade anterior é transmitida através das hipóteses.

O objecto `Pmf` tem dois atributos:

* "qs" contém as quantidades na distribuição;

* 'ps' contém as probabilidades correspondentes.

In [36]:
prior.qs

In [37]:
prior.ps

Agora estamos prontos para fazer a actualização.



In [38]:
likelihood1 = 1/6, 1/8, 1/12

E aqui está a actualização.

In [39]:
posterior = prior * likelihood1
posterior.normalize()
posterior



Aqui estão as probabilidades:

In [40]:
likelihood2 = 0, 1/8, 1/12

A probabilidade para o dado de 6 lados é 0 porque não é possível obter um 7 num dado de 6 lados.

As outras duas probabilidades são as mesmas que na actualização anterior.

Aqui está a actualização:

In [41]:
posterior *= likelihood2
posterior.normalize()
posterior

Depois de rodar um 1 e um 7, a probabilidade posterior do dado de 8 lados é de cerca de 69%.

## Actualização de dados



In [42]:
def update_dice(pmf, data):
    """Update pmf based on new data."""
    hypos = pmf.qs
    likelihood = 1 / hypos
    impossible = (data > hypos)
    likelihood[impossible] = 0
    pmf *= likelihood
    pmf.normalize()

O primeiro parâmetro é um `Pmf` que representa os possíveis dados e as suas probabilidades.

O segundo parâmetro é o resultado de enrolar um dado.

A primeira linha selecciona quantidades do `Pmf` que representam as hipóteses.

Uma vez que as hipóteses são inteiros, podemos utilizá-las para calcular as probabilidades.

Em geral, se houver "n" lados no dado, a probabilidade de qualquer resultado possível é de "1/n".

No entanto, temos de verificar os resultados impossíveis!

Se o resultado exceder o número hipotético de lados do coto, a probabilidade desse resultado é 0.

"Impossível" é uma "Série" Booleana que é "Verdadeira" para cada resultado impossível.

Utilizo-o como um índice em "probabilidade" para fixar as probabilidades correspondentes em 0.

Finalmente, multiplico `pmf` pelas probabilidades e normalizo.

Eis como podemos utilizar esta função para calcular as actualizações na secção anterior.

Começo com uma nova cópia da distribuição prévia:



In [43]:
pmf = prior.copy()
pmf

E utilizar `update_dice` para fazer as actualizações.

In [44]:
update_dice(pmf, 1)
update_dice(pmf, 7)
pmf

O resultado é o mesmo.  Veremos uma versão desta função no próximo capítulo.

## Resumo

Este capítulo introduz o módulo `empiricaldista', que fornece `Pmf', que utilizamos para representar um conjunto de hipóteses e as suas probabilidades.

O `empiricaldist' é baseado em Pandas; a classe `Pmf' herda da classe Pandas `Série' e fornece características adicionais específicas às funções de massa probabilística.

Vamos utilizar `Pmf` e outras classes de `empiricaldist` em todo o livro porque simplificam o código e tornam-no mais legível.

Mas poderíamos fazer as mesmas coisas directamente com Pandas.

Utilizamos um `Pmf` para resolver o problema dos biscoitos e dos dados, o que vimos no capítulo anterior.

Com um `Pmf` é fácil de realizar actualizações sequenciais com múltiplos dados.

Também resolvemos uma versão mais geral do problema das bolachas, com 101 taças em vez de duas.

Depois calculamos o MAP, que é a quantidade com a probabilidade posterior mais alta.

No próximo capítulo, introduzirei o problema do euro, e utilizaremos a distribuição binomial.

E, finalmente, daremos o salto de usar o Teorema de Bayes para fazer estatísticas Bayesianas.

Mas primeiro pode querer trabalhar nos exercícios.

## Exercícios

**Exercício:** Suponha que tenho uma caixa com um dado de 6 faces, um dado de 8 faces, e um dado de 12 faces.

Escolho um dos dados ao acaso, lanço-o quatro vezes, e recebo 1, 3, 5, e 7.

Qual é a probabilidade de eu ter escolhido o dado de 8 lados?

Pode utilizar a função `update_dice` ou fazer você mesmo a actualização.

In [45]:
# Solution goes here

**Exercício:** Na versão anterior do problema dos dados, as probabilidades anteriores são as mesmas porque a caixa contém um de cada dado.

Mas suponha que a caixa contém 1 dado de 4 faces, 2 dados de 6 faces, 3 dados de 8 faces, 4 dados de 12 faces, e 5 dados de 20 faces.

Escolho um dado, rolo-o, e recebo um 7.

Qual é a probabilidade de eu ter escolhido um dado de 8 lados?

Dica: Para fazer a distribuição prévia, chamar `Pmf` com dois parâmetros.

In [46]:
# Solution goes here

**Exercício:** Suponha que tenho duas gavetas de meias.

Uma contém números iguais de meias pretas e brancas.

A outra contém números iguais de meias vermelhas, verdes e azuis.

Suponhamos que eu escolho uma gaveta ao acaso, escolho duas meias ao acaso, e digo-vos que tenho um par a condizer.

Qual é a probabilidade de que as meias sejam brancas?

Para simplificar, vamos supor que existem tantas meias em ambas as gavetas que a remoção de uma meia faz uma mudança insignificante nas proporções.

In [47]:
# Solution goes here

In [48]:
# Solution goes here

**Exercício:** Aqui está um problema de [Bayesian Data Analysis](http://www.stat.columbia.edu/~gelman/book/):

> Elvis Presley tinha um irmão gémeo (que morreu à nascença). Qual é a probabilidade de Elvis ter sido um gémeo idêntico?

Dica: Em 1935, cerca de 2/3 dos gémeos eram fraternais e 1/3 eram idênticos.

In [49]:
# Solution goes here

In [50]:
# Solution goes here

In [51]:
# Solution goes here